<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-加载数据" data-toc-modified-id="1.-加载数据-1">1. 加载数据</a></span></li><li><span><a href="#2.-数据清洗" data-toc-modified-id="2.-数据清洗-2">2. 数据清洗</a></span></li><li><span><a href="#3.-特征构造" data-toc-modified-id="3.-特征构造-3">3. 特征构造</a></span></li><li><span><a href="#4.-数据拆分" data-toc-modified-id="4.-数据拆分-4">4. 数据拆分</a></span></li><li><span><a href="#5.-模型训练" data-toc-modified-id="5.-模型训练-5">5. 模型训练</a></span><ul class="toc-item"><li><span><a href="#5.1-LR" data-toc-modified-id="5.1-LR-5.1">5.1 LR</a></span></li><li><span><a href="#5.2-tree" data-toc-modified-id="5.2-tree-5.2">5.2 tree</a></span></li><li><span><a href="#5.3-GBDT" data-toc-modified-id="5.3-GBDT-5.3">5.3 GBDT</a></span></li><li><span><a href="#5.4-GBDT+LR" data-toc-modified-id="5.4-GBDT+LR-5.4">5.4 GBDT+LR</a></span></li></ul></li><li><span><a href="#6.-结果比较" data-toc-modified-id="6.-结果比较-6">6. 结果比较</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier,  RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 1. 加载数据

In [2]:
data_train = pd.read_csv('./used_car_train_20200313.csv', sep=' ')
data_submit = pd.read_csv('./used_car_testA_20200313.csv', sep=' ')

In [3]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             150000 non-null  int64  
 1   name               150000 non-null  int64  
 2   regDate            150000 non-null  int64  
 3   model              149999 non-null  float64
 4   brand              150000 non-null  int64  
 5   bodyType           145494 non-null  float64
 6   fuelType           141320 non-null  float64
 7   gearbox            144019 non-null  float64
 8   power              150000 non-null  int64  
 9   kilometer          150000 non-null  float64
 10  notRepairedDamage  150000 non-null  object 
 11  regionCode         150000 non-null  int64  
 12  seller             150000 non-null  int64  
 13  offerType          150000 non-null  int64  
 14  creatDate          150000 non-null  int64  
 15  price              150000 non-null  int64  
 16  v_

In [4]:
data_submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SaleID             50000 non-null  int64  
 1   name               50000 non-null  int64  
 2   regDate            50000 non-null  int64  
 3   model              50000 non-null  float64
 4   brand              50000 non-null  int64  
 5   bodyType           48587 non-null  float64
 6   fuelType           47107 non-null  float64
 7   gearbox            48090 non-null  float64
 8   power              50000 non-null  int64  
 9   kilometer          50000 non-null  float64
 10  notRepairedDamage  50000 non-null  object 
 11  regionCode         50000 non-null  int64  
 12  seller             50000 non-null  int64  
 13  offerType          50000 non-null  int64  
 14  creatDate          50000 non-null  int64  
 15  v_0                50000 non-null  float64
 16  v_1                500

In [5]:
# 变量类型
date_columns        = ['creatDate', 'regDate']
useless_columns     = ['SaleID', 'name', 'seller', 'offerType']
label_columns       = ['price']
categorical_columns = ['brand', 'bodyType', 'fuelType', 'gearbox', 
                       'notRepairedDamage', 'model','regionCode']
numeric_columns     = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 
                       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 
                       'v_10', 'v_11', 'v_12','v_13', 'v_14']

In [6]:
# 训练集数据
features = data_train[numeric_columns + categorical_columns + date_columns + label_columns]
labels = data_train[label_columns]

In [7]:
# 测试集数据
sub_features = data_submit[numeric_columns + categorical_columns + date_columns]

## 2. 数据清洗

In [8]:
# 非法值
features['notRepairedDamage'] = features['notRepairedDamage'].apply(lambda x: int(float(x)) if x != '-' else 0)
sub_features['notRepairedDamage'] = sub_features['notRepairedDamage'].apply(lambda x: int(float(x)) if x != '-' else 0)

In [9]:
# 缺失值(均值、众数补全)
null_columns = ['bodyType', 'fuelType', 'gearbox', 'model']
for col in null_columns:
    fill_value = features[col].mode().values[0]
    features.fillna(value={col: fill_value}, inplace=True)
    sub_features.fillna(value={col: fill_value}, inplace=True)

## 3. 特征构造

In [10]:
# 时间特征
features['creatDate'] = pd.to_datetime(features['creatDate'], format='%Y%m%d')
features['regDate'] = features['regDate'].apply(lambda x: str(x))
features['regDate'] = features['regDate'].apply(lambda x: x[:4]+'01'+x[6:] if x[4:6] == '00' else x)
features['regDate'] = pd.to_datetime(features['regDate'], format='%Y%m%d')
features['new_date'] = features['creatDate'] - features['regDate']
features['new_date'] = features['new_date'].apply(lambda x: x.days)
features.drop(date_columns, axis=1, inplace=True)

sub_features['creatDate'] = pd.to_datetime(sub_features['creatDate'], format='%Y%m%d')
sub_features['regDate'] = sub_features['regDate'].apply(lambda x: str(x))
sub_features['regDate'] = sub_features['regDate'].apply(lambda x: x[:4]+'01'+x[6:] if x[4:6] == '00' else x)
sub_features['regDate'] = pd.to_datetime(sub_features['regDate'], format='%Y%m%d')
sub_features['new_date'] = sub_features['creatDate'] - sub_features['regDate']
sub_features['new_date'] = sub_features['new_date'].apply(lambda x: x.days)
sub_features.drop(date_columns, axis=1, inplace=True)

In [11]:
# 增加价格最大最小平均标准差
add_col = ['brand']
agg_method = ['max', 'min', 'mean', 'median']
for col in add_col:
    # 设置列名与获取列数据的字典形式
    col_name = ['{}_{}'.format(col, x) for x in agg_method]
    temp_dict = features.groupby(col)['price'].agg(agg_method).to_dict()
    
    # 建立列名
    features = pd.concat([features, pd.DataFrame(columns=col_name)], axis=1)    
    sub_features = pd.concat([sub_features, pd.DataFrame(columns=col_name)], axis=1)  
    
    # 填充数据
    for i, j in zip(col_name, temp_dict):
        features[i] = features[col].apply(lambda x: temp_dict[j][x])
        sub_features[i] = sub_features[col].apply(lambda x: temp_dict[j][x])

features.drop('price', axis=1, inplace=True)

In [12]:
# 独热编码
for col in ['brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage',]:
    # 训练独热编码
    onehot = OneHotEncoder()
    features_array = onehot.fit_transform(features[col].values.reshape(-1, 1))
    sub_features_array = onehot.transform(sub_features[col].values.reshape(-1, 1))
    
    # 列名设置
    col_name = ['{}_{}'.format(col,x) for x in range(features[col].nunique())]
    
    # 训练集填充
    df_temp = pd.DataFrame(features_array.toarray(), columns=col_name)
    features = pd.concat([features, df_temp], axis=1)
    features.drop(col, axis=1)
    
    # 测试集填充
    df_temp = pd.DataFrame(sub_features_array.toarray(), columns=col_name)
    sub_features = pd.concat([sub_features, df_temp], axis=1)
    sub_features.drop(col, axis=1)
    break

## 4. 数据拆分

In [13]:
ss = StandardScaler()
x = ss.fit_transform(features)
y = np.log(labels)
sub_train = ss.transform(sub_features)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1234)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(120000, 69)
(120000, 1)
(30000, 69)
(30000, 1)


## 5. 模型训练
### 5.1 LR

In [14]:
lr = LinearRegression()
lr.fit(x_train, y_train)
predict = lr.predict(x_test)
result_lr = np.exp(predict)
y_true = np.exp(y_test)
print('LR MAE:{}'.format(mean_absolute_error(result_lr, y_true)))

LR MAE:942.2024881889614


### 5.2 tree

In [15]:
tree = DecisionTreeRegressor()
tree.fit(x_train, y_train)
predict = tree.predict(x_test)
result_tree = np.exp(predict)
y_true = np.exp(y_test)
print('TREE MAE:{}'.format(mean_absolute_error(result_tree, y_true)))

TREE MAE:871.964958203364


### 5.3 GBDT

In [16]:
gbdt = GradientBoostingRegressor()
gbdt.fit(x_train, y_train)
predict = gbdt.predict(x_test)
result_gbdt = np.exp(predict)
y_true = np.exp(y_test)
print('GBDT MAE:{}'.format(mean_absolute_error(result_gbdt, y_true)))

GBDT MAE:871.0509874145639


### 5.4 GBDT+LR

In [17]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(x, y, test_size=0.5, random_state=666)
X_train_lr, X_train_tree, y_train_lr, y_train_tree = train_test_split(X_train_, y_train_, test_size=0.5, random_state=666)

# 训练GBDT
gbtd_lr = GradientBoostingRegressor(n_estimators=300)
gbtd_lr.fit(X_train_tree, y_train_tree)
temp = gbtd_lr.apply(X_train_tree)

# 训练一个onehot
onehot = OneHotEncoder()
onehot.fit(temp)

# 获得LR的features
lr_features = onehot.transform(gbtd_lr.apply(X_train_lr))

# 训练LR
reg_lr = LinearRegression()
reg_lr.fit(lr_features, y_train_lr)

# 模型预测
test_features = onehot.transform(gbtd_lr.apply(X_test_))
predict = reg_lr.predict(test_features)
result_gbdt_lr = np.exp(predict)
y_true = np.exp(y_test_)
print('GBDT MAE:{}'.format(mean_absolute_error(result_gbdt_lr, y_true)))

GBDT MAE:700.2495691824465


## 6. 结果比较

In [18]:
sub_csv = pd.read_csv('./submission.csv')
sub_csv.head()

,SaleID,price
0,150000,NaN
1,150001,NaN
2,150002,NaN
3,150003,NaN
4,150004,NaN


In [19]:
result_lr = np.exp(lr.predict(sub_train)) # lr 
result_tree = np.exp(tree.predict(sub_train)) # tree
result_GBDT = np.exp(gbdt.predict(sub_train)) # GBDT
sub_lr_features = onehot.transform(gbtd_lr.apply(sub_train))
result_GBDT_lr = np.exp(reg_lr.predict(sub_lr_features)) # GBDT+lr

sub_csv['result_lr'] = result_lr
sub_csv['result_tree'] = result_tree
sub_csv['result_GBDT'] = result_GBDT
sub_csv['result_GBDT_lr'] = result_GBDT_lr

sub_csv.to_csv('./model_predict.csv', index=False)
pd.read_csv('./model_predict.csv').head()

,SaleID,price,result_lr,result_tree,result_GBDT,result_GBDT_lr
0,150000,NaN,39151.056107,25500.0,30332.869136,31763.451079
1,150001,NaN,291.676860,350.0,309.252074,361.154800
2,150002,NaN,5918.867746,5700.0,6852.431879,6008.417314
3,150003,NaN,12272.146510,10500.0,12182.430788,11690.517126
4,150004,NaN,619.499519,650.0,651.270472,604.448705
